In [1]:
import sys
import os

# Go up one directory from /notebooks into repo root
repo_root = os.path.abspath("..")
if repo_root not in sys.path:
    sys.path.append(repo_root)

print("PYTHONPATH:", sys.path)

PYTHONPATH: ['c:\\Users\\prest\\.conda\\envs\\EchelonAcoustics312\\python312.zip', 'c:\\Users\\prest\\.conda\\envs\\EchelonAcoustics312\\DLLs', 'c:\\Users\\prest\\.conda\\envs\\EchelonAcoustics312\\Lib', 'c:\\Users\\prest\\.conda\\envs\\EchelonAcoustics312', '', 'c:\\Users\\prest\\.conda\\envs\\EchelonAcoustics312\\Lib\\site-packages', 'c:\\Users\\prest\\.conda\\envs\\EchelonAcoustics312\\Lib\\site-packages\\win32', 'c:\\Users\\prest\\.conda\\envs\\EchelonAcoustics312\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\prest\\.conda\\envs\\EchelonAcoustics312\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\prest\\OneDrive\\SAIT\\EchelonAcoustics']


In [2]:
import os
import torch
import librosa
import numpy as np
import json
import matplotlib.pyplot as plt
from backend.vgg_model import VGGSmall

MODEL_FILE = "../model/vgg_frog_model.pth"
LABEL_MAP_FILE = "../model/label_mapping.json"
TEST_DIR = "../data/frog_test_clips/"
TARGET_SR = 22050
N_MELS = 128

#### Load Model + Labels

In [3]:
#model = torch.load(MODEL_FILE, map_location="cpu")
model = VGGSmall()
model.load_state_dict(torch.load(MODEL_FILE, map_location="cpu"))
model.eval()

with open(LABEL_MAP_FILE, "r") as f:
    label_map = json.load(f)

#### Run Inference on Test Clips

In [4]:
def predict(path):
    y, sr = librosa.load(path, sr=TARGET_SR)
    y = y[:22050*5]  # trim to 5 sec

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS)
    S_dB = librosa.power_to_db(S, ref=np.max)
    S_dB = torch.tensor(S_dB).unsqueeze(0).unsqueeze(0).float()

    with torch.no_grad():
        logits = model(S_dB)
        probs = torch.softmax(logits, dim=1)
        conf, pred = torch.max(probs, dim=1)

    return label_map[str(int(pred.item()))], float(conf.item())

#### Test All Files

In [5]:
for f in os.listdir(TEST_DIR):
    path = os.path.join(TEST_DIR, f)
    species, conf = predict(path)
    print(f"{f}: {species} ({conf:.3f})")

bufo_boreas_western_toad.wav: frog (1.000)
bufo_cognatus_great_plains_toad.wav: not_frog (0.725)
bufo_hemiophrys_canadian_toad.wav: frog (1.000)
rana_pipiens_northern_leopard_frog.wav: not_frog (0.997)
rana_sylvatica_wood_frog.wav: frog (1.000)
